In [4]:
N = ord("В")% 5 +1
N

3

In [9]:
import pandas as pd

df = pd.read_excel("C:/Users/panis/OneDrive/Рабочий стол/year3/Python/Lab5/lab6.xlsx")

df.head()


,N,file name,image size,glasses color,line width
0,1,emma-watson2.jpg,300x300,red,2
1,2,emma-watson.jpg,400x400,blue,3
2,3,draco.jpg,700x700,cian,4
3,4,longbottom.jpg,600x600,magenta,5
4,5,ron_wesley.jpg,500x500,yellow,6


In [52]:
import os
import pandas as pd
import cv2
from PIL import Image, ImageDraw
import numpy as np

color_dict = {
    "red": (255, 0, 0),
    "blue": (0, 0, 255),
    "cian": (0, 255, 255),
    "magenta": (255, 0, 255),
    "green": (0, 255, 0),
    "yellow": (255, 255, 0),
    "black": (0, 0, 0),
    "white": (255, 255, 255)
}

excel_path = "C:/Users/panis/OneDrive/Рабочий стол/year3/Python/Lab5/lab6.xlsx"
image_folder = "C:/Users/panis/OneDrive/Рабочий стол/year3/Python/Lab5/Images"

def process_image(image_path, output_path, glasses_color, line_width, width, height):
    if not os.path.exists(image_path):
        print(f"Файл {image_path} не знайдено!")
        return

    try:
        image_pil = Image.open(image_path)
    except Exception as e:
        print(f"Помилка завантаження зображення {image_path}: {e}")
        return

    original_width, original_height = image_pil.size

    square_side = min(original_width, original_height)

    image = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    eyes_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

    # Знаходження обличчя
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))

    for (x, y, w, h) in faces:
        # Знаходимо центр обличчя
        face_center = (x + w // 2, y + h // 2)

        # Визначаємо координати верхнього лівого кута квадрату, щоб він був центрований на обличчі
        crop_x1 = max(face_center[0] - square_side // 2, 0)
        crop_y1 = max(face_center[1] - square_side // 2, 0)
        crop_x2 = min(face_center[0] + square_side // 2, original_width)
        crop_y2 = min(face_center[1] + square_side // 2, original_height)

        # Обрізаємо зображення до квадрата
        cropped_image = image_pil.crop((crop_x1, crop_y1, crop_x2, crop_y2))

        # Масштабування зображення до заданого розміру
        resized_image = cropped_image.resize((width, height))

        # Малювання очок після змінення розміру зображення
        draw = ImageDraw.Draw(resized_image)

        # Визначення координат очей для малювання окулярів
        # Використовуємо ті самі координати, що й раніше для знаходження очей на обрізаному зображенні

        gray_resized = cv2.cvtColor(np.array(resized_image), cv2.COLOR_RGB2GRAY)
        eyes = eyes_cascade.detectMultiScale(gray_resized)

        eye_centers = []
        for (ex, ey, ew, eh) in eyes:
            eye_centers.append((ex + ew // 2, ey + eh // 2))

        if len(eye_centers) == 2:
            # Визначаємо порядок очей за x-координатою
            eye_centers.sort(key=lambda p: p[0])

            # Малювання круглих окулярів
            radius = 40
            for i, center in enumerate(eye_centers):
                draw.ellipse([center[0] - radius, center[1] - radius, center[0] + radius, center[1] + radius],
                             outline=glasses_color, width=line_width)

                if i == 0:  # Ліве око
                    draw.line([center[0] - radius, center[1],
                               center[0] - 2 * radius, center[1]],
                              fill=glasses_color, width=line_width)
                if i == 1:  # Праве око
                    draw.line([center[0] + radius, center[1],
                               center[0] + 2 * radius, center[1]],
                              fill=glasses_color, width=line_width)

            draw.line([eye_centers[0][0] + radius, eye_centers[0][1],
                       eye_centers[1][0] - radius, eye_centers[1][1]],
                      fill=glasses_color, width=line_width)

        # Збереження зображення
        resized_image.save(output_path)
        print(f"Оброблене зображення збережено: {output_path}")
        return

    print("Обличчя не знайдено!")

# Завантаження даних з Excel
df = pd.read_excel(excel_path)

variant = 3  # Ваш номер варіанту
row = df[df['N'] == variant].iloc[0]  # Пошук рядка за номером варіанту

file_name = row['file name'].strip()
glasses_color_name = row['glasses color'].strip().lower()
print(glasses_color_name)
glasses_color = color_dict.get(glasses_color_name)
line_width = int(row['line width'])

image_size = row['image size'].strip()
width, height = map(int, image_size.split('x'))  # Розмір зображення

# Шляхи до вхідного та вихідного зображень
input_image_path = os.path.join(image_folder, file_name)
output_image_path = os.path.join(image_folder, f"output_{file_name}")

# Виклик функції обробки зображення
process_image(input_image_path, output_image_path, glasses_color, line_width, width, height)


cian
Оброблене зображення збережено: C:/Users/panis/OneDrive/Рабочий стол/year3/Python/Lab5/Images\output_draco.jpg


In [15]:
import os

image_path = r"C:\Users\panis\OneDrive\Рабочий стол\year3\Python\Lab5\Images\draco.jpg"
if os.path.exists(image_path):
    print("Файл існує!")
else:
    print("Файл не знайдено!")


Файл існує!
